In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
from gurobi import *

# Classes

In [81]:
class Driver(object):
    def __init__(self, courier_ID, shift_start, shift_end, MoT, start_lon, start_lat):
        self.courier_ID = courier_ID
        self.shift_start = shift_start
        self.shift_end = shift_end
        self.MoT = MoT
        self.start_lon = start_lon
        self.start_lat = start_lat
        
class Job(object):
    def __init__(self, job_ID, job_kind, pickup_lon, pickup_lat, delivery_lon, delivery_lat):
        self.job_ID = job_ID
        self.job_kind = job_kind
        self.pickup_lon = pickup_lon
        self.pickup_lat = pickup_lat
        self.delivery_lon = delivery_lon
        self.delivery_lat = delivery_lat
        
    def travel_time(self, MoT):
        return self.pickup.travel_time(self, delivery_lon, deliver_lat)
        
#need to review class position    
class Position(object):
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat
        self.dist = {}
        
    def add(self, position, time):
        self.dist[position] = time
        
    def travel_time(self, position_lon, position_lat):
        return self.dist[position]

# Travel Matrix

In [69]:
path = 'Travel_Matrix.csv'
inputdata = pd.read_csv(path)

In [70]:
inputdata.head()

,start_lon,start_lat,pickup_lon,pickup_lat,MoT,distance_meters,time_seconds
0,-0.077893,51.50963,-0.182688,51.494267,scooter,8169.1,772.6
1,-0.077893,51.50963,-0.124112,51.378221,car,17908.4,1851.4
2,-0.077893,51.50963,-0.070889,51.473192,bicycle,5893.9,1541.0
3,-0.077893,51.50963,-0.171205,51.469288,scooter,9917.6,935.0
4,-0.077893,51.50963,-0.328978,51.447641,car,20977.3,1911.8


### Lists

In [72]:
start_lon = [inputdata['start_lon']]
start_lat = [inputdata['start_lat']]
pickup_lon = [inputdata['pickup_lon']]
pickup_lat = [inputdata['pickup_lat']]
MoT = [inputdata['MoT']]
distance_meters = [inputdata['distance_meters']]
time_seconds = [inputdata['time_seconds']]

# Jobs data

In [76]:
path = 'Jobsdataforselectedday (1).csv'
inputdata = pd.read_csv(path)

In [77]:
inputdata.head()

,encr_job_id,job_state,job_kind,b2b_delivery_restrictions,started_at,assigned_at,items_ready_at,pickup_arrived_at,drop_started_at,drop_arrived_at,job_completed_at,cancelled_at,disg_courier_id,transport_mode,encr_pickup_id,pickup_lat,pickup_lon,offset_drop_lat,offset_drop_lon
0,41e715a1a8401966d459e92e502018e1,complete,B2B scheduled,NaN,10/06/2019 07:00,50:03.0,NaN,53:37.0,46:52.0,52:21.0,57:24.0,NaN,112,Car,eb19fae70183e190ed302cfc384346ae,51.485816,-0.180878,51.494596,-0.175581
1,047b08657758ac8c0131158145e3ebaf,complete,B2C on demand,NaN,05:25.0,06:03.0,NaN,15:18.0,27:25.0,34:07.0,36:19.0,NaN,14,Scooter,6a7afe87251855e7b2939e87d6934034,51.515897,-0.204797,51.520986,-0.166067
2,6e398ce770b25f4383af626bd0d66d02,complete,B2C on demand,NaN,06:52.0,11:40.0,NaN,27:55.0,38:20.2,39:18.3,51:45.0,NaN,148,Scooter,d5651d9abc4a757b40ac3dcf70a837e0,51.535891,-0.205417,51.535138,-0.211440
3,dd2e3b82ea1b32cee629b13620e92fdc,complete,B2C on demand,NaN,17:51.0,19:03.0,NaN,19:28.0,33:56.7,35:42.5,39:54.0,NaN,40,Bicycle,939f71635ccbb759c05ffd8dea59fee6,51.489974,-0.162701,51.492780,-0.166601
4,4c672bfed253e6199321f50ff7b87090,complete,B2C on demand,NaN,20:22.0,22:04.0,NaN,35:25.8,50:08.6,55:09.6,57:54.0,NaN,13,Scooter,0ceb8f47864a220f07157cff0ed18b9a,51.521655,-0.134368,51.531419,-0.120289


### Lists

In [78]:
job_ID = [inputdata['encr_job_id']]
job_kind = [inputdata['job_kind']]
delivery_lon = [inputdata['offset_drop_lon']]
delivery_lat = [inputdata['offset_drop_lat']]

# Shift data 

In [60]:
path = 'Shiftsdataforselectedday (1).csv'
inputdata = pd.read_csv(path)

In [61]:
inputdata.head()

,disg_shift_id,disg_courier_id,shift_start_time,shift_end_time,starting_lon,starting_lat
0,1,139,10/06/2019 11:00,10/06/2019 21:00,-0.076223,51.524128
1,2,150,10/06/2019 11:00,10/06/2019 21:00,-0.123455,51.515900
2,3,151,10/06/2019 11:05,10/06/2019 21:05,-0.076223,51.524128
3,4,163,10/06/2019 14:00,10/06/2019 17:06,-0.163983,51.475052
4,5,175,10/06/2019 11:00,10/06/2019 21:15,-0.163983,51.475052


### Lists

In [62]:
courier_ID = [inputdata['disg_courier_id']]
shift_start = [inputdata['shift_start_time']]
shift_end = [inputdata['shift_end_time']]

### Parameters

In [11]:
#Parameters
pickup_timeout_bike_scooter = 30
delivery_timeout_bike_scooter = 30
pickup_timeout_car = 60
delivery_timeout_car = 60
SLA_start_to_pickup = 15 #time only taken into account while driver is on the move
SLA_pickup_to_delivery = 15 #it does not take into account waiting time at pickup of driver
SLA_satisfy = 0.95 #90%-95% of SLAs must be satisfied --> 0.05 instead of 0.95?
gm_raduis = 5000 #5000m radius search to allocate driver to job